# Imports

In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.io as spio
import matplotlib.pyplot as plt

from pymor.models.iosys import LTIModel

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

# Build model

In [ ]:
mat = spio.loadmat('data/ABCE.mat')
mat.keys()

In [ ]:
mu = np.sqrt(10) * np.array([0.2, 0.4, 0.6, 0.8])
A = mat['A0']
for i in range(4):
    A += mu[i] * mat[f'A{i + 1}']
B = mat['B']
C = mat['C']
E = mat['E']

In [ ]:
fom = LTIModel.from_matrices(A, B, C, E=E)

In [ ]:
fom

In [ ]:
print(fom)

# Bode plot

In [ ]:
w = np.logspace(-2, 4, 100)
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = fom.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = fom.mag_plot(w)

# $\mathcal{H}_2$ norm

$$
\newcommand{\tran}{^{\operatorname{T}}}
\begin{align*}
  A P E\tran
  + E P A\tran
  + B B\tran
  & = 0 \\
  A\tran Q E
  + E\tran Q A
  + C\tran C
  & = 0
\end{align*}
$$

$$
\Vert H \Vert_{\mathcal{H}_2}^2
= \operatorname{tr}\left(C P C\tran\right)
= \operatorname{tr}\left(B\tran Q B\right)
$$

In [ ]:
fom.h2_norm()

# Bitangential Hermite interpolation

Full-order model:
$$
\begin{align*}
  E \dot{x}(t) & = A x(t) + B u(t), \\
  y(t) & = C x(t).
\end{align*}
$$

Reduced-order model:
$$
\begin{align*}
  \hat{E} \dot{\hat{x}}(t) & = \hat{A} \hat{x}(t) + \hat{B} u(t), \\
  \hat{y}(t) & = \hat{C} \hat{x}(t),
\end{align*}
$$
where
$$
\begin{align*}
  \hat{E} = W\tran E V, \quad
  \hat{A} = W\tran A V, \quad
  \hat{B} = W\tran B, \quad
  \hat{C} = C V.
\end{align*}
$$

Transfer functions:
$$
\begin{align*}
  H(s) & = C (s E - A)^{-1} B, \\
  \hat{H}(s) & = \hat{C} \left(s \hat{E} - \hat{A}\right)^{-1} \hat{B}.
\end{align*}
$$

Theorem 1 from [Antoulas/Beattie/Gugercin 2010]:
$$
\begin{align*}
  (\sigma E - A)^{-1} B b \in \operatorname{im}(V)
  & \quad \Rightarrow \quad
  H(\sigma) b = \hat{H}(\sigma) b, \\
  (\sigma E - A)^{-*} C^{\operatorname{T}} c \in \operatorname{im}(W)
  & \quad \Rightarrow \quad
  c^* H(\sigma) = c^* \hat{H}(\sigma), \\
  \text{both}
  & \quad \Rightarrow \quad
  c^* H'(\sigma) b = c^* \hat{H}'(\sigma) b.
\end{align*}
$$

In [ ]:
from pymor.reductors.interpolation import LTIBHIReductor

In [ ]:
sigma = [10j, -10j]
b = np.array([[1], [1]])
b = fom.B.source.from_numpy(b)
c = np.array([[1, 0, 0, 0], [1, 0, 0, 0]])
c = fom.C.range.from_numpy(c)

In [ ]:
interp = LTIBHIReductor(fom)

In [ ]:
rom_interp = interp.reduce(sigma, b, c)

In [ ]:
rom_interp

In [ ]:
print(rom_interp)

In [ ]:
fom.eval_tf(10j)

In [ ]:
rom_interp.eval_tf(10j)

In [ ]:
fom.eval_tf(10j) - rom_interp.eval_tf(10j)

In [ ]:
fom.eval_dtf(10j)

In [ ]:
fom.eval_dtf(10j) - rom_interp.eval_dtf(10j)

In [ ]:
err_interp = fom - rom_interp

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = err_interp.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = err_interp.mag_plot(w)

In [ ]:
err_interp.h2_norm() / fom.h2_norm()

# Iterative rational Krylov algorithm (IRKA)

Interpolatory necessary conditions for $\mathcal{H}_2$-optimality
[Meier/Luenberger 1967, Antoulas/Beattie/Gugercin 2006/2008/2010]:
If $\hat{H}(s) = \sum_{i = 1}^r \frac{c_i b_i^*}{s - \lambda_i}$ is an
$\mathcal{H}_2$-optimal ROM for $H$,
then
$$
\begin{align*}
  H(-\overline{\lambda_i}) b_i
  & = \hat{H}(-\overline{\lambda_i}) b_i, \\
  c_i^* H(-\overline{\lambda_i})
  & = c_i^* \hat{H}(-\overline{\lambda_i}), \\
  c_i^* H'(-\overline{\lambda_i}) b_i
  & = c_i^* \hat{H}'(-\overline{\lambda_i}) b_i.
\end{align*}
$$

In [ ]:
from pymor.reductors.h2 import IRKAReductor

In [ ]:
irka = IRKAReductor(fom)

In [ ]:
rom_irka = irka.reduce(10)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(irka.conv_crit, '.-')
ax.set_xlabel('Iteration number')
_ = ax.set_title('Convergence criterion')

In [ ]:
from matplotlib import animation

In [ ]:
plt.rcParams['animation.html'] = 'jshtml'

In [ ]:
fig, ax = plt.subplots()

sigmas = np.concatenate(irka.sigma_list)
s_re_min = sigmas.real.min()
s_re_max = sigmas.real.max()
s_im_max = sigmas.imag.max()

ax.set_xlim((s_re_min, s_re_max))
ax.set_ylim((-s_im_max, s_im_max))
ax.set_xscale('symlog')
ax.set_yscale('symlog')

line, = ax.plot([], [], '.')

In [ ]:
def init():
    line.set_data([], [])
    return (line,)

def animate(i):
    ax.set_title(f'Iteration {i + 1}')
    line.set_data(irka.sigma_list[i].real, irka.sigma_list[i].imag)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(irka.sigma_list), interval=500, blit=True)

anim

In [ ]:
irka_poles = rom_irka.poles()
_ = plt.plot(irka_poles.real, irka_poles.imag, '.')

In [ ]:
err_irka = fom - rom_irka

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = err_irka.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = err_irka.mag_plot(w)

In [ ]:
err_irka.h2_norm() / fom.h2_norm()

# Two-sided iteration algorithm (TSIA)

Gramian-based necessary conditions for $\mathcal{H}_2$-optimality [Wilson 1960]:
$$
\begin{align*}
  \hat{Q} \hat{E} \hat{P} & = Y\tran E X, \\
  \hat{Q} \hat{A} \hat{P} & = Y\tran A X, \\
  \hat{Q} \hat{B} & = Y\tran B, \\
  \hat{C} \hat{P} & = C X,
\end{align*}
$$
where
$$
\begin{align*}
  A X \hat{E}\tran
  + E X \hat{A}\tran
  + B \hat{B}\tran
  & = 0, \\
  A\tran Y \hat{E}
  + E\tran Y \hat{A}
  + C\tran \hat{C}
  & = 0, \\
  \hat{A} \hat{P} \hat{E}\tran
  + \hat{E} \hat{P} \hat{A}\tran
  + \hat{B} \hat{B}\tran
  & = 0, \\
  \hat{A}\tran \hat{Q} \hat{E}
  + \hat{E}\tran \hat{Q} \hat{A}
  + \hat{C}\tran \hat{C}
  & = 0.
\end{align*}
$$

In [ ]:
from pymor.reductors.h2 import TSIAReductor

In [ ]:
tsia = TSIAReductor(fom)

In [ ]:
rom_tsia = tsia.reduce(10)

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(tsia.conv_crit, '.-')
ax.set_xlabel('Iteration number')
_ = ax.set_title('Convergence criterion')

In [ ]:
tsia_poles = rom_tsia.poles()
_ = plt.plot(tsia_poles.real, tsia_poles.imag, '.')

In [ ]:
err_tsia = fom - rom_tsia

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), constrained_layout=True)
_ = err_tsia.bode_plot(w, ax=np.array(4 * [[axs[0]], [axs[1]]]))

In [ ]:
_ = err_tsia.mag_plot(w)

In [ ]:
err_tsia.h2_norm() / fom.h2_norm()

# Comparison with balanced truncation

In [ ]:
from pymor.reductors.bt import BTReductor

In [ ]:
bt = BTReductor(fom)
rom_bt = bt.reduce(10)
err_bt = fom - rom_bt

In [ ]:
err_bt.h2_norm() / fom.h2_norm()

# Related methods

- Structure-preserving interpolation:
  - second-order models (`SOBHIReductor`)
  \begin{align*}
    &
    \left\{
    \begin{aligned}
      M \ddot{x}(t) + E \dot{x}(t) + K x(t) & = B u(t), \\
      y(t) & = C_p x(t) + C_v \dot{x}(t)
    \end{aligned}
    \right. \\
    &
    H(s) = (C_p + s C_v) \left(s^2 M + s E + K\right)^{-1} B
  \end{align*}
  - time-delay models (`DelayBHIReductor`)
  \begin{align*}
    &
    \left\{
    \begin{aligned}
      E \dot{x}(t) & = A_0 x(t) + A_\tau x(t - \tau) + B u(t), \\
      y(t) & = C x(t)
    \end{aligned}
    \right. \\
    &
    H(s) = C \left(s E - A_0 - e^{-\tau s} A_\tau\right)^{-1} B
  \end{align*}
- Transfer function approach using Loewner matrices (`TFBHIReductor`)
- Symmetry-preserving IRKA (`OneSidedIRKAReductor`)
- IRKA-type method for second-order models (`SORIRKAReductor`)
- Transfer function IRKA (`TFIRKAReductor`)